In [2]:
import torch
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import os
%matplotlib inline


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path = '/content/gdrive/MyDrive/lesson1_dataset'

In [5]:
def read_files(X, Y, path, ans):
  files = os.listdir(path)
  for name in files:
    img = cv2.imread(path + '/' + name, 0)
    if img.shape != 0:
      img = cv2.resize(img, (256, 256))
      vect = img.reshape(1, 256 ** 2)
      vect = vect / 255.
      X = vect if (X is None) else np.vstack((X, vect)) 
      Y = np.append(Y, ans)
  return X, Y

In [6]:
X = None
Y = np.array([])
X, Y = read_files(X, Y, path + "/logloss_0", 0)
X, Y = read_files(X, Y, path + "/logloss_1", 1)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.30, random_state=42)

In [8]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, X,Y):
        self.X = X
        self.Y = Y

  def __len__(self):
    #num = sum(os.path.isfile(os.path.join(self.path, f)) for f in os.listdir(self.path + "/logloss_0"))
    return len(self.X)

  def __getitem__(self, index):
    x1 = torch.tensor(self.X[index], dtype = torch.float32)
    y1 = torch.tensor(self.Y[index], dtype = torch.float32)
    return (x1,y1)

In [9]:
train_set = Dataset(X_train,Y_train)

Loader_train = torch.utils.data.DataLoader(train_set,batch_size=128, shuffle=True)

In [10]:
test_set = Dataset(X_test,Y_test)

Loader_test = torch.utils.data.DataLoader(test_set,batch_size=128, shuffle=True)

In [11]:
for x,y in Loader_train:
  print(x.view(x.shape[0], -1).shape, y.shape)
  break

torch.Size([45, 65536]) torch.Size([45])


In [12]:
for x,y in Loader_test:
  print(x.view(x.shape[0], -1).shape, y.shape)
  break

torch.Size([20, 65536]) torch.Size([20])


In [13]:
import torch.nn as nn

In [14]:
model = nn.Sequential(
    nn.Linear(65536, 5000),
    nn.BatchNorm1d(5000),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(5000, 2000),
    nn.BatchNorm1d(2000),
    nn.ReLU(),
    nn.Linear(2000, 2000),
    nn.BatchNorm1d(2000),
    nn.ReLU(),
    nn.Linear(2000,2000),
    nn.BatchNorm1d(2000),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(2000, 10),
    nn.Softmax()
)

In [15]:
model.cuda()

Sequential(
  (0): Linear(in_features=65536, out_features=5000, bias=True)
  (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): ReLU()
  (4): Linear(in_features=5000, out_features=2000, bias=True)
  (5): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Linear(in_features=2000, out_features=2000, bias=True)
  (8): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU()
  (10): Linear(in_features=2000, out_features=2000, bias=True)
  (11): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Dropout(p=0.5, inplace=False)
  (13): ReLU()
  (14): Linear(in_features=2000, out_features=10, bias=True)
  (15): Softmax(dim=None)
)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [18]:
epochs = 20
for i in range(epochs):
    for j, (x, y) in enumerate(Loader_train):
        model.train()
        optimizer.zero_grad()
        x = x.view(x.shape[0], -1)
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x)
        loss = criterion(y_pred, y.long())
        accuracy = (torch.argmax(y_pred.detach().cpu(),1)== y.cpu()).sum()/len(y)
        print("TRAIN")
        print(f"Epoch {i}\t iter {j}\t loss {loss}\t accuracy {accuracy}")
        loss.backward()
        optimizer.step()
    with torch.no_grad():
      for j, (x, y) in enumerate(Loader_test):
            model.eval()
            optimizer.zero_grad()
            x = x.cuda()
            y = y.cuda()
            y_pred = model(x)
            loss = criterion(y_pred, y.long())
            accuracy = (torch.argmax(y_pred.detach().cpu(),1)== y.cpu()).sum()/len(y)
            print("TEST")
            print(f"Epoch {i}\t iter {j}\t loss {loss}\t test_accuracy {accuracy}")
            optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


TRAIN
Epoch 0	 iter 0	 loss 1.4613724946975708	 accuracy 1.0
TEST
Epoch 0	 iter 0	 loss 1.6455729007720947	 test_accuracy 0.800000011920929
TRAIN
Epoch 1	 iter 0	 loss 1.461212158203125	 accuracy 1.0
TEST
Epoch 1	 iter 0	 loss 1.6337506771087646	 test_accuracy 0.800000011920929
TRAIN
Epoch 2	 iter 0	 loss 1.4612118005752563	 accuracy 1.0
TEST
Epoch 2	 iter 0	 loss 1.6041405200958252	 test_accuracy 0.8500000238418579
TRAIN
Epoch 3	 iter 0	 loss 1.461253046989441	 accuracy 1.0
TEST
Epoch 3	 iter 0	 loss 1.5867446660995483	 test_accuracy 0.8999999761581421
TRAIN
Epoch 4	 iter 0	 loss 1.4611880779266357	 accuracy 1.0
TEST
Epoch 4	 iter 0	 loss 1.5820271968841553	 test_accuracy 0.8999999761581421
TRAIN
Epoch 5	 iter 0	 loss 1.4611903429031372	 accuracy 1.0
TEST
Epoch 5	 iter 0	 loss 1.5805165767669678	 test_accuracy 0.8999999761581421
TRAIN
Epoch 6	 iter 0	 loss 1.46120023727417	 accuracy 1.0
TEST
Epoch 6	 iter 0	 loss 1.5794689655303955	 test_accuracy 0.8999999761581421
TRAIN
Epoch 7	 iter